<a href="https://colab.research.google.com/github/hi-wesley/skyrl-demo/blob/main/SkyRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -LsSf https://astral.sh/uv/install.sh | sh
import os
os.environ["PATH"] = os.path.expanduser("~/.local/bin") + ":" + os.environ["PATH"]
!uv --version

downloading uv 0.9.13 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
uv 0.9.13


In [2]:
!git clone https://github.com/NovaSky-AI/SkyRL.git
%cd SkyRL/skyrl-train

Cloning into 'SkyRL'...
remote: Enumerating objects: 7476, done.
remote: Counting objects: 100% (374/374), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 7476 (delta 274), reused 188 (delta 188), pack-reused 7102 (from 4)
Receiving objects: 100% (7476/7476), 14.00 MiB | 20.84 MiB/s, done.
Resolving deltas: 100% (4496/4496), done.
/content/SkyRL/skyrl-train


In [3]:
import os
os.environ["RAY_RUNTIME_ENV_HOOK"] = "ray._private.runtime_env.uv_runtime_env_hook.hook"

In [4]:
# still in SkyRL/skyrl-train
!uv run --isolated examples/gsm8k/gsm8k_dataset.py --output_dir /content/data/gsm8k

Installed 122 packages in 260ms
README.md: 7.94kB [00:00, 33.6MB/s]
main/train-00000-of-00001.parquet: 100% 2.31M/2.31M [00:00<00:00, 5.60MB/s]
main/test-00000-of-00001.parquet: 100% 419k/419k [00:00<00:00, 2.40MB/s]
Generating train split: 100% 7473/7473 [00:00<00:00, 525298.46 examples/s]
Generating test split: 100% 1319/1319 [00:00<00:00, 347439.99 examples/s]
Map: 100% 7473/7473 [00:00<00:00, 18084.95 examples/s]
Map: 100% 1319/1319 [00:00<00:00, 19631.68 examples/s]
Creating parquet from Arrow format: 100% 8/8 [00:00<00:00, 211.83ba/s]
Creating parquet from Arrow format: 100% 2/2 [00:00<00:00, 284.96ba/s]


In [5]:
!uv run --isolated -- python -m ray start --head

Installed 122 packages in 263ms
/root/.cache/uv/builds-v0/.tmp9NtOfN/bin/python: No module named ray.__main__; 'ray' is a package and cannot be directly executed


In [6]:
%%bash
cat << 'EOF' > run_gsm8k_colab.sh
set -x

DATA_DIR="/content/data/gsm8k"
NUM_GPUS=1
LOGGER="console"
INFERENCE_BACKEND="vllm"

uv run --isolated --extra ${INFERENCE_BACKEND} -m skyrl_train.entrypoints.main_base \
  data.train_data="['${DATA_DIR}/train.parquet']" \
  data.val_data="['${DATA_DIR}/validation.parquet']" \
  trainer.algorithm.advantage_estimator="grpo" \
  trainer.policy.model.path="Qwen/Qwen2.5-0.5B-Instruct" \
  trainer.strategy="fsdp2" \
  trainer.placement.colocate_all=true \
  trainer.placement.policy_num_gpus_per_node=${NUM_GPUS} \
  trainer.placement.ref_num_gpus_per_node=${NUM_GPUS} \
  generator.backend="${INFERENCE_BACKEND}" \
  generator.run_engines_locally=true \
  generator.weight_sync_backend="nccl" \
  generator.num_inference_engines=${NUM_GPUS} \
  generator.inference_engine_tensor_parallel_size=1 \
  generator.async_engine=true \
  generator.batched=true \
  generator.n_samples_per_prompt=2 \
  generator.gpu_memory_utilization=0.8 \
  trainer.epochs=1 \
  trainer.train_batch_size=32 \
  trainer.policy_mini_batch_size=8 \
  trainer.micro_forward_batch_size_per_gpu=4 \
  trainer.micro_train_batch_size_per_gpu=4 \
  trainer.eval_batch_size=64 \
  trainer.eval_before_train=false \
  trainer.eval_interval=50 \
  trainer.max_prompt_length=512 \
  generator.sampling_params.max_generate_length=256 \
  trainer.policy.optimizer_config.lr=1e-6 \
  trainer.algorithm.use_kl_loss=true \
  environment.env_class="gsm8k" \
  trainer.logger="${LOGGER}" \
  trainer.project_name="gsm8k" \
  trainer.run_name="gsm8k_colab_test" \
  trainer.resume_mode=null \
  trainer.ckpt_path="/content/ckpts/gsm8k_0_5B" \
  "$@"
EOF

chmod +x run_gsm8k_colab.sh

In [7]:
!bash run_gsm8k_colab.sh

Streaming output truncated to the last 5000 lines.
Policy Train epoch [1/1]:  94%|█████████▍| 15/16 [00:06<00:00,  2.37it/s, pg=0, glen=256, policy_lr=1e-6, ent=0.228]
(skyrl_entrypoint pid=5568) 2025-11-28 04:59:39.281 | INFO     | skyrl_train.trainer:train_critic_and_policy:897 - Finished: 'policy_train', time cost: 7.63s
Policy Train epoch [1/1]: 100%|██████████| 16/16 [00:07<00:00,  2.25it/s, pg=0, glen=256, policy_lr=1e-6, ent=0.0921, grad_norm=2.52]
(skyrl_entrypoint pid=5568) 2025-11-28 04:59:39.347 | INFO     | skyrl_train.trainer:train:250 - Finished: 'train_critic_and_policy', time cost: 7.70s
(skyrl_entrypoint pid=5568) 2025-11-28 04:59:39.756 | INFO     | skyrl_train.trainer:train:278 - Started: 'sync_weights'
(AsyncVLLMInferenceEngine pid=5936) (EngineCore_DP0 pid=6105) INFO 11-28 04:59:39 [executor_base.py:205] It took 0.004336 seconds to wake up tags ['weights'].
(AsyncVLLMInferenceEngine pid=5936) (EngineCore_DP0 pid=6105) INFO 11-28 04:59:40 [block_pool.py:378] Success